In [6]:
from sentence_transformers import SentenceTransformer, models
import torch, faiss, numpy as np

# --- Modell A: Fertiger Sentence-Transformer ---
model_a = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")
def embed_a(texts):
    return np.array(model_a.encode(texts, convert_to_numpy=True, normalize_embeddings=True))

# --- Modell B: Legal-BERT als Sentence-Transformer mit Pooling ---
word_embedding_model = models.Transformer("nlpaueb/legal-bert-base-uncased")
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
model_b = SentenceTransformer(modules=[word_embedding_model, pooling_model])
def embed_b(texts):
    return np.array(model_b.encode(texts, convert_to_numpy=True, normalize_embeddings=True))

# --- Modell C: Juristisch optimierter Sentence-Transformer ---
model_c = SentenceTransformer("Stern5497/sbert-legal-xlm-roberta-base")
def embed_c(texts):
    return np.array(model_c.encode(texts, convert_to_numpy=True, normalize_embeddings=True))

# --- Gesetzestexte ---
laws = [
    "§ 1 BGB: Die Rechtsfähigkeit des Menschen beginnt mit der Vollendung der Geburt.",
    "Art. 5 GG: Jeder hat das Recht, seine Meinung in Wort, Schrift und Bild frei zu äußern.",
    "DSGVO Art. 6: Verarbeitung personenbezogener Daten ist nur rechtmäßig, wenn...",
    "Artikel 13 StGG: Jedermann hat das Recht, durch Wort, Schrift, Druck oder durch bildliche Darstellung seine Meinung innerhalb der gesetzlichen Schranken frei zu äußern.",
    "Artikel 10 EMRK: Jede Person hat das Recht auf freie Meinungsäußerung."
]

# --- Prompt Eingabe ---
prompt = input("Bitte gib deine Anfrage ein: ")

# --- Funktion für Top-1 Treffer ---
def retrieve_top1(embed_fn, model_name, laws, query):
    emb = embed_fn(laws)
    dim = emb.shape[1]
    index = faiss.IndexFlatIP(dim)
    index.add(emb)

    qvec = embed_fn([query])
    D, I = index.search(qvec, k=1)
    idx, score = I[0][0], D[0][0]
    print(f"{model_name} -> Top Treffer: {laws[idx]} (Score={score:.3f})")

# --- Ausgabe für alle Modelle ---
print("\nTop-1 Treffer für deinen Prompt:")
retrieve_top1(embed_a, "all-mpnet-base-v2", laws, prompt)
retrieve_top1(embed_b, "legal-bert (SentenceTransformer)", laws, prompt)
retrieve_top1(embed_c, "sbert-legal-xlm-roberta-base", laws, prompt)

Bitte gib deine Anfrage ein:  meinungsfreiheit



Top-1 Treffer für deinen Prompt:
all-mpnet-base-v2 -> Top Treffer: Artikel 10 EMRK: Jede Person hat das Recht auf freie Meinungsäußerung. (Score=0.547)
legal-bert (SentenceTransformer) -> Top Treffer: Art. 5 GG: Jeder hat das Recht, seine Meinung in Wort, Schrift und Bild frei zu äußern. (Score=0.685)
sbert-legal-xlm-roberta-base -> Top Treffer: Artikel 10 EMRK: Jede Person hat das Recht auf freie Meinungsäußerung. (Score=0.864)


In [2]:
def evaluate(embed_fn, laws, queries, k=3):
    emb = embed_fn(laws)
    dim = emb.shape[1]
    index = faiss.IndexFlatIP(dim)
    index.add(emb)

    reciprocal_ranks, correct_at_k = [], 0
    for query, expected_idx in queries:
        qvec = embed_fn([query])
        D, I = index.search(qvec, k=k)
        if expected_idx in I[0]:
            correct_at_k += 1
            rank = list(I[0]).index(expected_idx) + 1
            reciprocal_ranks.append(1.0 / rank)
        else:
            reciprocal_ranks.append(0.0)

    return correct_at_k / len(queries), np.mean(reciprocal_ranks)

# --- Ground Truth Queries ---
queries = [
    ("Meinungsfreiheit in Deutschland", 1),
    ("Meinungsfreiheit in Österreich", 3),
    ("Datenschutz personenbezogene Daten", 2)
]

# --- Ergebnisse ---
acc_a, mrr_a = evaluate(embed_a, laws, queries)
acc_b, mrr_b = evaluate(embed_b, laws, queries)
acc_c, mrr_c = evaluate(embed_c, laws, queries)

print("\nVergleich:")
print(f"all-mpnet-base-v2 -> Accuracy@3: {acc_a:.3f}, MRR: {mrr_a:.3f}")
print(f"legal-bert (SentenceTransformer) -> Accuracy@3: {acc_b:.3f}, MRR: {mrr_b:.3f}")
print(f"sbert-legal-xlm-roberta-base -> Accuracy@3: {acc_c:.3f}, MRR: {mrr_c:.3f}")


Vergleich:
all-mpnet-base-v2 -> Accuracy@3: 0.667, MRR: 0.500
legal-bert (SentenceTransformer) -> Accuracy@3: 1.000, MRR: 0.667
sbert-legal-xlm-roberta-base -> Accuracy@3: 1.000, MRR: 0.611


In [3]:
import pandas as pd

def evaluate_with_top3_errors(embed_fn, model_name, laws, queries, k=3):
    emb = embed_fn(laws)
    dim = emb.shape[1]
    index = faiss.IndexFlatIP(dim)
    index.add(emb)

    rows = []
    for query, expected_idx in queries:
        qvec = embed_fn([query])
        D, I = index.search(qvec, k=3)   # nur Top-3 Treffer

        for rank, (idx, score) in enumerate(zip(I[0], D[0]), 1):
            correct = (idx == expected_idx)
            reason = ""
            if not correct:
                if "Meinungsfreiheit" in laws[idx] and "Meinungsfreiheit" in laws[expected_idx]:
                    reason = "Lexikalische Überlappung"
                elif "Daten" in laws[idx] and "DSGVO" in laws[expected_idx]:
                    reason = "Domain-Mismatch"
                else:
                    reason = "Allgemeine semantische Verwechslung"
            rows.append({
                "Query": query,
                "Model": model_name,
                "Expected": laws[expected_idx],
                "Retrieved": laws[idx],
                "Rank": rank,
                "Score": round(float(score), 3),
                "Correct": correct,
                "Reason": reason
            })
    return pd.DataFrame(rows)

# --- Fehleranalyse für alle Modelle (nur Top-3) ---
df_a = evaluate_with_top3_errors(embed_a, "all-mpnet-base-v2", laws, queries)
df_b = evaluate_with_top3_errors(embed_b, "legal-bert (Mean Pooling)", laws, queries)
df_c = evaluate_with_top3_errors(embed_c, "sbert-legal-xlm-roberta-base", laws, queries)

# Zusammenführen
df_all = pd.concat([df_a, df_b, df_c], ignore_index=True)

# In Jupyter einfach df_all ausgeben:
df_all

,Query,Model,Expected,Retrieved,Rank,Score,Correct,Reason
0,Meinungsfreiheit in Deutschland,all-mpnet-base-v2,"Art. 5 GG: Jeder hat das Recht, seine Meinung ...",Artikel 10 EMRK: Jede Person hat das Recht auf...,1,0.532,False,Allgemeine semantische Verwechslung
1,Meinungsfreiheit in Deutschland,all-mpnet-base-v2,"Art. 5 GG: Jeder hat das Recht, seine Meinung ...","Art. 5 GG: Jeder hat das Recht, seine Meinung ...",2,0.520,True,
2,Meinungsfreiheit in Deutschland,all-mpnet-base-v2,"Art. 5 GG: Jeder hat das Recht, seine Meinung ...",DSGVO Art. 6: Verarbeitung personenbezogener D...,3,0.410,False,Allgemeine semantische Verwechslung
3,Meinungsfreiheit in Österreich,all-mpnet-base-v2,"Artikel 13 StGG: Jedermann hat das Recht, durc...",Artikel 10 EMRK: Jede Person hat das Recht auf...,1,0.577,False,Allgemeine semantische Verwechslung
4,Meinungsfreiheit in Österreich,all-mpnet-base-v2,"Artikel 13 StGG: Jedermann hat das Recht, durc...","Art. 5 GG: Jeder hat das Recht, seine Meinung ...",2,0.485,False,Allgemeine semantische Verwechslung
5,Meinungsfreiheit in Österreich,all-mpnet-base-v2,"Artikel 13 StGG: Jedermann hat das Recht, durc...",DSGVO Art. 6: Verarbeitung personenbezogener D...,3,0.388,False,Allgemeine semantische Verwechslung
6,Datenschutz personenbezogene Daten,all-mpnet-base-v2,DSGVO Art. 6: Verarbeitung personenbezogener D...,DSGVO Art. 6: Verarbeitung personenbezogener D...,1,0.669,True,
7,Datenschutz personenbezogene Daten,all-mpnet-base-v2,DSGVO Art. 6: Verarbeitung personenbezogener D...,Artikel 10 EMRK: Jede Person hat das Recht auf...,2,0.402,False,Allgemeine semantische Verwechslung
8,Datenschutz personenbezogene Daten,all-mpnet-base-v2,DSGVO Art. 6: Verarbeitung personenbezogener D...,§ 1 BGB: Die Rechtsfähigkeit des Menschen begi...,3,0.377,False,Allgemeine semantische Verwechslung
9,Meinungsfreiheit in Deutschland,legal-bert (Mean Pooling),"Art. 5 GG: Jeder hat das Recht, seine Meinung ...",§ 1 BGB: Die Rechtsfähigkeit des Menschen begi...,1,0.797,False,Allgemeine semantische Verwechslung


In [4]:
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

# --- Gesetzestexte ---
laws = [
    "§ 1 BGB: Die Rechtsfähigkeit des Menschen beginnt mit der Vollendung der Geburt.",
    "Art. 5 GG: Jeder hat das Recht, seine Meinung in Wort, Schrift und Bild frei zu äußern.",
    "DSGVO Art. 6: Verarbeitung personenbezogener Daten ist nur rechtmäßig, wenn...",
    "Artikel 13 StGG: Jedermann hat das Recht, durch Wort, Schrift, Druck oder durch bildliche Darstellung seine Meinung innerhalb der gesetzlichen Schranken frei zu äußern.",
    "Artikel 10 EMRK: Jede Person hat das Recht auf freie Meinungsäußerung."
]

# --- Training examples (query, law, label) ---
train_examples = [
    InputExample(texts=["Meinungsfreiheit in Deutschland", laws[1]], label=1.0),
    InputExample(texts=["Meinungsfreiheit in Deutschland", laws[3]], label=0.0),
    InputExample(texts=["Meinungsfreiheit in Österreich", laws[3]], label=1.0),
    InputExample(texts=["Datenschutz personenbezogene Daten", laws[2]], label=1.0),
    InputExample(texts=["Datenschutz personenbezogene Daten", laws[1]], label=0.0),
]
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)

# --- Training function ---
def train_model(model_name, output_dir):
    print(f"\n=== Training {model_name} ===")
    model = SentenceTransformer(model_name)
    train_loss = losses.CosineSimilarityLoss(model)
    model.fit(
        train_objectives=[(train_dataloader, train_loss)],
        epochs=2,
        warmup_steps=100,
        show_progress_bar=True,
        output_path=output_dir
    )
    print(f"Saved fine-tuned model to {output_dir}")

# --- Train all three models ---
train_model("sentence-transformers/all-mpnet-base-v2", "output/mpnet-legal")
train_model("nlpaueb/legal-bert-base-uncased", "output/legal-bert")
train_model("Stern5497/sbert-legal-xlm-roberta-base", "output/legal-sbert")


=== Training sentence-transformers/all-mpnet-base-v2 ===


C:\Users\micha\AppData\Local\Python\pythoncore-3.14-64\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss


No sentence-transformers model found with name nlpaueb/legal-bert-base-uncased. Creating a new one with mean pooling.


Saved fine-tuned model to output/mpnet-legal

=== Training nlpaueb/legal-bert-base-uncased ===


Step,Training Loss


Saved fine-tuned model to output/legal-bert

=== Training Stern5497/sbert-legal-xlm-roberta-base ===


Step,Training Loss


Saved fine-tuned model to output/legal-sbert


In [5]:
import faiss, numpy as np
from sentence_transformers import SentenceTransformer

# --- Evaluation corpus ---
laws = [
    "§ 1 BGB: Die Rechtsfähigkeit des Menschen beginnt mit der Vollendung der Geburt.",
    "Art. 5 GG: Jeder hat das Recht, seine Meinung in Wort, Schrift und Bild frei zu äußern.",
    "DSGVO Art. 6: Verarbeitung personenbezogener Daten ist nur rechtmäßig, wenn...",
    "Artikel 13 StGG: Jedermann hat das Recht, durch Wort, Schrift, Druck oder durch bildliche Darstellung seine Meinung innerhalb der gesetzlichen Schranken frei zu äußern.",
    "Artikel 10 EMRK: Jede Person hat das Recht auf freie Meinungsäußerung."
]

queries = [
    ("Meinungsfreiheit in Deutschland", 1),
    ("Meinungsfreiheit in Österreich", 3),
    ("Datenschutz personenbezogene Daten", 2)
]

# --- Evaluation function ---
def evaluate(model, laws, queries, k=3):
    emb = np.array(model.encode(laws, convert_to_numpy=True, normalize_embeddings=True))
    dim = emb.shape[1]
    index = faiss.IndexFlatIP(dim)
    index.add(emb)

    reciprocal_ranks, correct_at_k = [], 0
    for query, expected_idx in queries:
        qvec = np.array(model.encode([query], convert_to_numpy=True, normalize_embeddings=True))
        D, I = index.search(qvec, k=k)
        if expected_idx in I[0]:
            correct_at_k += 1
            rank = list(I[0]).index(expected_idx) + 1
            reciprocal_ranks.append(1.0 / rank)
        else:
            reciprocal_ranks.append(0.0)

    return correct_at_k / len(queries), np.mean(reciprocal_ranks)

# --- Evaluate all fine-tuned models ---
for name, path in [
    ("all-mpnet-base-v2", "output/mpnet-legal"),
    ("legal-bert-base-uncased", "output/legal-bert"),
    ("sbert-legal-xlm-roberta-base", "output/legal-sbert")
]:
    model = SentenceTransformer(path)
    acc, mrr = evaluate(model, laws, queries)
    print(f"{name} -> Accuracy@3: {acc:.3f}, MRR: {mrr:.3f}")

all-mpnet-base-v2 -> Accuracy@3: 0.667, MRR: 0.500
legal-bert-base-uncased -> Accuracy@3: 1.000, MRR: 0.667
sbert-legal-xlm-roberta-base -> Accuracy@3: 1.000, MRR: 0.611
